# Oracle Table Diagnostics
## WH_LOANS, WH_ACCTCOMMON, WH_ACCT

Identifies:
1. Date columns with values outside .NET range (causes CopyJob DateTime errors)
2. Number/Decimal precision issues
3. Sample problematic rows

In [4]:
# ============================================================
# CONNECTION CONFIGURATION - UPDATE THESE VALUES
# ============================================================

ORACLE_HOST = "10.1.102.53:24625/1625dm"  # Oracle server hostname
ORACLE_PORT = "1521"                                      # Usually 1521
ORACLE_SERVICE = "COCCDM"                                 # Service name or SID
ORACLE_USER = "smith1625"                             # Oracle username
ORACLE_PASSWORD = "v0c4l1z3!"                         # Oracle password

# Connection string format (update if using different format)
# TNS format: ORACLE_DSN = "(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST={host})(PORT={port}))(CONNECT_DATA=(SERVICE_NAME={service})))"
ORACLE_DSN = "10.1.102.53:24625/1625dm"

SCHEMA = "COCCDM"

In [5]:
# Install oracledb if needed (uncomment)
!pip install oracledb pandas

^C


In [7]:
import oracledb
import pandas as pd
from datetime import datetime

# Try thin mode first (no Oracle client needed)
 # Comment this out if using thin mode

print(f"Connecting to {ORACLE_DSN} as {ORACLE_USER}...")

try:
    conn = oracledb.connect(
        user=ORACLE_USER,
        password=ORACLE_PASSWORD,
        dsn=ORACLE_DSN
    )
    print(f"Connected successfully!")
    print(f"Oracle version: {conn.version}")
except Exception as e:
    print(f"Connection failed: {e}")
    print("\nTry thin mode by commenting out oracledb.init_oracle_client()")

Connecting to 10.1.102.53:24625/1625dm as smith1625...
Connected successfully!
Oracle version: 19.26.0.0.0


In [8]:
def run_query(sql, desc=""):
    """Run SQL and return DataFrame"""
    print(f"\n{'='*60}")
    print(f"{desc}")
    print(f"{'='*60}")
    try:
        df = pd.read_sql(sql, conn)
        print(df.to_string())
        return df
    except Exception as e:
        print(f"ERROR: {e}")
        return None

## 1. Row Counts

In [9]:
run_query("""
SELECT 'WH_LOANS' as table_name, COUNT(*) as row_count FROM COCCDM.WH_LOANS
UNION ALL
SELECT 'WH_ACCTCOMMON', COUNT(*) FROM COCCDM.WH_ACCTCOMMON
UNION ALL
SELECT 'WH_ACCT', COUNT(*) FROM COCCDM.WH_ACCT
""", "TABLE ROW COUNTS")


TABLE ROW COUNTS


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


      TABLE_NAME  ROW_COUNT
0       WH_LOANS  119050162
1  WH_ACCTCOMMON  254461030
2        WH_ACCT  262386697


,TABLE_NAME,ROW_COUNT
0,WH_LOANS,119050162
1,WH_ACCTCOMMON,254461030
2,WH_ACCT,262386697


## 2. Date Range Check (Normal Dates)

In [10]:
run_query("""
SELECT 
    'WH_LOANS' as tbl,
    MIN(RUNDATE) as min_rundate,
    MAX(RUNDATE) as max_rundate,
    COUNT(DISTINCT RUNDATE) as distinct_dates
FROM COCCDM.WH_LOANS
""", "WH_LOANS - RUNDATE Range")


WH_LOANS - RUNDATE Range


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


        TBL MIN_RUNDATE MAX_RUNDATE  DISTINCT_DATES
0  WH_LOANS  2018-08-06  2025-12-01            1837


,TBL,MIN_RUNDATE,MAX_RUNDATE,DISTINCT_DATES
0,WH_LOANS,2018-08-06,2025-12-01,1837


In [11]:
run_query("""
SELECT 
    'WH_ACCTCOMMON' as tbl,
    MIN(EFFDATE) as min_effdate,
    MAX(EFFDATE) as max_effdate,
    COUNT(DISTINCT EFFDATE) as distinct_dates
FROM COCCDM.WH_ACCTCOMMON
""", "WH_ACCTCOMMON - EFFDATE Range")


WH_ACCTCOMMON - EFFDATE Range


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


             TBL MIN_EFFDATE MAX_EFFDATE  DISTINCT_DATES
0  WH_ACCTCOMMON  2018-08-06  2025-12-01            1837


,TBL,MIN_EFFDATE,MAX_EFFDATE,DISTINCT_DATES
0,WH_ACCTCOMMON,2018-08-06,2025-12-01,1837


In [12]:
run_query("""
SELECT 
    'WH_ACCT' as tbl,
    MIN(RUNDATE) as min_rundate,
    MAX(RUNDATE) as max_rundate,
    COUNT(DISTINCT RUNDATE) as distinct_dates
FROM COCCDM.WH_ACCT
""", "WH_ACCT - RUNDATE Range")


WH_ACCT - RUNDATE Range


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


       TBL MIN_RUNDATE MAX_RUNDATE  DISTINCT_DATES
0  WH_ACCT  2018-08-06  2025-12-01            1837


,TBL,MIN_RUNDATE,MAX_RUNDATE,DISTINCT_DATES
0,WH_ACCT,2018-08-06,2025-12-01,1837


## 3. INVALID DATE CHECK - WH_LOANS (The Problem Table)
Dates outside .NET range (< 0001-01-01 or > 9999-12-31) cause CopyJob failures

In [13]:
# Check each date column in WH_LOANS for invalid dates
date_columns_loans = ['RUNDATE', 'ORIGDATE', 'DATEMAT', 'PAIDOFFDATE', 'DATELASTMAINT', 'ADDDATE', 'STOPDATE']

print("WH_LOANS - INVALID DATE CHECK")
print("="*70)

for col in date_columns_loans:
    try:
        df = pd.read_sql(f"""
            SELECT 
                '{col}' as date_column,
                COUNT(*) as invalid_count,
                MIN({col}) as min_date,
                MAX({col}) as max_date
            FROM COCCDM.WH_LOANS
            WHERE EXTRACT(YEAR FROM {col}) < 1 
               OR EXTRACT(YEAR FROM {col}) > 9999
               OR {col} < TO_DATE('0001-01-01', 'YYYY-MM-DD')
        """, conn)
        invalid = df['INVALID_COUNT'].iloc[0]
        if invalid > 0:
            print(f"\n*** {col}: {invalid} INVALID ROWS ***")
            print(df.to_string())
        else:
            print(f"{col}: OK (0 invalid)")
    except Exception as e:
        print(f"{col}: ERROR - {e}")

WH_LOANS - INVALID DATE CHECK


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2384887917.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"""


RUNDATE: OK (0 invalid)
ORIGDATE: OK (0 invalid)
DATEMAT: ERROR - Execution failed on sql '
            SELECT 
                'DATEMAT' as date_column,
                COUNT(*) as invalid_count,
                MIN(DATEMAT) as min_date,
                MAX(DATEMAT) as max_date
            FROM COCCDM.WH_LOANS
            WHERE EXTRACT(YEAR FROM DATEMAT) < 1 
               OR EXTRACT(YEAR FROM DATEMAT) > 9999
               OR DATEMAT < TO_DATE('0001-01-01', 'YYYY-MM-DD')
        ': ORA-00904: "DATEMAT": invalid identifier
Help: https://docs.oracle.com/error-help/db/ora-00904/
PAIDOFFDATE: ERROR - Execution failed on sql '
            SELECT 
                'PAIDOFFDATE' as date_column,
                COUNT(*) as invalid_count,
                MIN(PAIDOFFDATE) as min_date,
                MAX(PAIDOFFDATE) as max_date
            FROM COCCDM.WH_LOANS
            WHERE EXTRACT(YEAR FROM PAIDOFFDATE) < 1 
               OR EXTRACT(YEAR FROM PAIDOFFDATE) > 9999
               OR PAIDOF

In [14]:
# Find extreme dates (outside reasonable range 1900-2100)
run_query("""
SELECT 
    'ORIGDATE' as col,
    COUNT(*) as extreme_count,
    MIN(ORIGDATE) as min_val,
    MAX(ORIGDATE) as max_val
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM ORIGDATE) < 1900 OR EXTRACT(YEAR FROM ORIGDATE) > 2100
UNION ALL
SELECT 'DATEMAT', COUNT(*), MIN(DATEMAT), MAX(DATEMAT)
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM DATEMAT) < 1900 OR EXTRACT(YEAR FROM DATEMAT) > 2100
UNION ALL
SELECT 'PAIDOFFDATE', COUNT(*), MIN(PAIDOFFDATE), MAX(PAIDOFFDATE)
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM PAIDOFFDATE) < 1900 OR EXTRACT(YEAR FROM PAIDOFFDATE) > 2100
""", "WH_LOANS - EXTREME DATES (outside 1900-2100)")


WH_LOANS - EXTREME DATES (outside 1900-2100)
ERROR: Execution failed on sql '
SELECT 
    'ORIGDATE' as col,
    COUNT(*) as extreme_count,
    MIN(ORIGDATE) as min_val,
    MAX(ORIGDATE) as max_val
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM ORIGDATE) < 1900 OR EXTRACT(YEAR FROM ORIGDATE) > 2100
UNION ALL
SELECT 'DATEMAT', COUNT(*), MIN(DATEMAT), MAX(DATEMAT)
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM DATEMAT) < 1900 OR EXTRACT(YEAR FROM DATEMAT) > 2100
UNION ALL
SELECT 'PAIDOFFDATE', COUNT(*), MIN(PAIDOFFDATE), MAX(PAIDOFFDATE)
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM PAIDOFFDATE) < 1900 OR EXTRACT(YEAR FROM PAIDOFFDATE) > 2100
': ORA-00904: "DATEMAT": invalid identifier
Help: https://docs.oracle.com/error-help/db/ora-00904/


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


In [15]:
# Sample rows with problematic dates
run_query("""
SELECT ACCTNBR, RUNDATE, ORIGDATE, DATEMAT, PAIDOFFDATE, STATUS, ORIGBAL
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM ORIGDATE) < 1900 
   OR EXTRACT(YEAR FROM ORIGDATE) > 2100
   OR EXTRACT(YEAR FROM DATEMAT) < 1900
   OR EXTRACT(YEAR FROM DATEMAT) > 2100
FETCH FIRST 20 ROWS ONLY
""", "WH_LOANS - SAMPLE ROWS WITH BAD DATES")


WH_LOANS - SAMPLE ROWS WITH BAD DATES
ERROR: Execution failed on sql '
SELECT ACCTNBR, RUNDATE, ORIGDATE, DATEMAT, PAIDOFFDATE, STATUS, ORIGBAL
FROM COCCDM.WH_LOANS
WHERE EXTRACT(YEAR FROM ORIGDATE) < 1900 
   OR EXTRACT(YEAR FROM ORIGDATE) > 2100
   OR EXTRACT(YEAR FROM DATEMAT) < 1900
   OR EXTRACT(YEAR FROM DATEMAT) > 2100
FETCH FIRST 20 ROWS ONLY
': ORA-00904: "DATEMAT": invalid identifier
Help: https://docs.oracle.com/error-help/db/ora-00904/


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


## 4. INVALID DATE CHECK - WH_ACCTCOMMON

In [16]:
date_columns_acctcommon = ['EFFDATE', 'CONTRACTDATE', 'DATEMAT', 'CLOSEDATE', 'DATELASTMAINT']

print("WH_ACCTCOMMON - INVALID DATE CHECK")
print("="*70)

for col in date_columns_acctcommon:
    try:
        df = pd.read_sql(f"""
            SELECT 
                '{col}' as date_column,
                COUNT(*) as invalid_count,
                MIN({col}) as min_date,
                MAX({col}) as max_date
            FROM COCCDM.WH_ACCTCOMMON
            WHERE EXTRACT(YEAR FROM {col}) < 1 
               OR EXTRACT(YEAR FROM {col}) > 9999
        """, conn)
        invalid = df['INVALID_COUNT'].iloc[0]
        if invalid > 0:
            print(f"\n*** {col}: {invalid} INVALID ROWS ***")
            print(df.to_string())
        else:
            print(f"{col}: OK (0 invalid)")
    except Exception as e:
        print(f"{col}: ERROR - {e}")

WH_ACCTCOMMON - INVALID DATE CHECK


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2698724775.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"""


EFFDATE: OK (0 invalid)
CONTRACTDATE: OK (0 invalid)
DATEMAT: OK (0 invalid)
CLOSEDATE: OK (0 invalid)
DATELASTMAINT: OK (0 invalid)


## 5. INVALID DATE CHECK - WH_ACCT

In [17]:
date_columns_acct = ['RUNDATE', 'DATEMAT', 'EFFDATE', 'DATELASTMAINT']

print("WH_ACCT - INVALID DATE CHECK")
print("="*70)

for col in date_columns_acct:
    try:
        df = pd.read_sql(f"""
            SELECT 
                '{col}' as date_column,
                COUNT(*) as invalid_count,
                MIN({col}) as min_date,
                MAX({col}) as max_date
            FROM COCCDM.WH_ACCT
            WHERE EXTRACT(YEAR FROM {col}) < 1 
               OR EXTRACT(YEAR FROM {col}) > 9999
        """, conn)
        invalid = df['INVALID_COUNT'].iloc[0]
        if invalid > 0:
            print(f"\n*** {col}: {invalid} INVALID ROWS ***")
            print(df.to_string())
        else:
            print(f"{col}: OK (0 invalid)")
    except Exception as e:
        print(f"{col}: ERROR - {e}")

WH_ACCT - INVALID DATE CHECK


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2876915434.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"""


RUNDATE: OK (0 invalid)
DATEMAT: OK (0 invalid)
EFFDATE: ERROR - Execution failed on sql '
            SELECT 
                'EFFDATE' as date_column,
                COUNT(*) as invalid_count,
                MIN(EFFDATE) as min_date,
                MAX(EFFDATE) as max_date
            FROM COCCDM.WH_ACCT
            WHERE EXTRACT(YEAR FROM EFFDATE) < 1 
               OR EXTRACT(YEAR FROM EFFDATE) > 9999
        ': ORA-00904: "EFFDATE": invalid identifier
Help: https://docs.oracle.com/error-help/db/ora-00904/
DATELASTMAINT: OK (0 invalid)


## 6. NUMERIC PRECISION CHECK
Check column definitions for NUMBER precision/scale

In [18]:
run_query("""
SELECT table_name, column_name, data_type, data_precision, data_scale, nullable
FROM all_tab_columns
WHERE owner = 'COCCDM'
  AND table_name = 'WH_LOANS'
  AND data_type = 'NUMBER'
ORDER BY column_id
""", "WH_LOANS - NUMBER COLUMN DEFINITIONS")


WH_LOANS - NUMBER COLUMN DEFINITIONS
   TABLE_NAME         COLUMN_NAME DATA_TYPE  DATA_PRECISION  DATA_SCALE NULLABLE
0    WH_LOANS             ACCTNBR    NUMBER              22           0        N
1    WH_LOANS             ORIGBAL    NUMBER              22           0        Y
2    WH_LOANS            CURRTERM    NUMBER              22           0        Y
3    WH_LOANS                INTC    NUMBER              22           0        Y
4    WH_LOANS               DELYR    NUMBER              22           0        Y
5    WH_LOANS             DELLIFE    NUMBER              22           0        Y
6    WH_LOANS               OLDPI    NUMBER              22           2        Y
7    WH_LOANS             ORIGINT    NUMBER              22           7        Y
8    WH_LOANS          REMAINTERM    NUMBER              22           0        Y
9    WH_LOANS                SOLD    NUMBER              24          10        Y
10   WH_LOANS             SOLDBAL    NUMBER              22           2

C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,TABLE_NAME,COLUMN_NAME,DATA_TYPE,DATA_PRECISION,DATA_SCALE,NULLABLE
0,WH_LOANS,ACCTNBR,NUMBER,22,0,N
1,WH_LOANS,ORIGBAL,NUMBER,22,0,Y
2,WH_LOANS,CURRTERM,NUMBER,22,0,Y
3,WH_LOANS,INTC,NUMBER,22,0,Y
4,WH_LOANS,DELYR,NUMBER,22,0,Y
5,WH_LOANS,DELLIFE,NUMBER,22,0,Y
6,WH_LOANS,OLDPI,NUMBER,22,2,Y
7,WH_LOANS,ORIGINT,NUMBER,22,7,Y
8,WH_LOANS,REMAINTERM,NUMBER,22,0,Y
9,WH_LOANS,SOLD,NUMBER,24,10,Y


In [19]:
run_query("""
SELECT table_name, column_name, data_type, data_precision, data_scale, nullable
FROM all_tab_columns
WHERE owner = 'COCCDM'
  AND table_name = 'WH_ACCTCOMMON'
  AND data_type = 'NUMBER'
ORDER BY column_id
""", "WH_ACCTCOMMON - NUMBER COLUMN DEFINITIONS")


WH_ACCTCOMMON - NUMBER COLUMN DEFINITIONS


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


       TABLE_NAME          COLUMN_NAME DATA_TYPE  DATA_PRECISION  DATA_SCALE NULLABLE
0   WH_ACCTCOMMON              ACCTNBR    NUMBER              22           0        N
1   WH_ACCTCOMMON         BRANCHORGNBR    NUMBER              22           0        N
2   WH_ACCTCOMMON           BANKORGNBR    NUMBER              22           0        N
3   WH_ACCTCOMMON          NOTEINTRATE    NUMBER               8           7        Y
4   WH_ACCTCOMMON          NOTEOPENAMT    NUMBER              15           2        Y
5   WH_ACCTCOMMON              NOTEBAL    NUMBER              15           2        Y
6   WH_ACCTCOMMON          BOOKBALANCE    NUMBER              15           2        Y
7   WH_ACCTCOMMON        NOTEMTDAVGBAL    NUMBER              15           2        Y
8   WH_ACCTCOMMON  NOTEINTCALCSCHEDNBR    NUMBER              22           0        Y
9   WH_ACCTCOMMON              INTBASE    NUMBER               8           0        Y
10  WH_ACCTCOMMON         INTMINBALAMT    NUMBER      

,TABLE_NAME,COLUMN_NAME,DATA_TYPE,DATA_PRECISION,DATA_SCALE,NULLABLE
0,WH_ACCTCOMMON,ACCTNBR,NUMBER,22,0,N
1,WH_ACCTCOMMON,BRANCHORGNBR,NUMBER,22,0,N
2,WH_ACCTCOMMON,BANKORGNBR,NUMBER,22,0,N
3,WH_ACCTCOMMON,NOTEINTRATE,NUMBER,8,7,Y
4,WH_ACCTCOMMON,NOTEOPENAMT,NUMBER,15,2,Y
5,WH_ACCTCOMMON,NOTEBAL,NUMBER,15,2,Y
6,WH_ACCTCOMMON,BOOKBALANCE,NUMBER,15,2,Y
7,WH_ACCTCOMMON,NOTEMTDAVGBAL,NUMBER,15,2,Y
8,WH_ACCTCOMMON,NOTEINTCALCSCHEDNBR,NUMBER,22,0,Y
9,WH_ACCTCOMMON,INTBASE,NUMBER,8,0,Y


In [20]:
run_query("""
SELECT table_name, column_name, data_type, data_precision, data_scale, nullable
FROM all_tab_columns
WHERE owner = 'COCCDM'
  AND table_name = 'WH_ACCT'
  AND data_type = 'NUMBER'
ORDER BY column_id
""", "WH_ACCT - NUMBER COLUMN DEFINITIONS")

C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)



WH_ACCT - NUMBER COLUMN DEFINITIONS
   TABLE_NAME        COLUMN_NAME DATA_TYPE  DATA_PRECISION  DATA_SCALE NULLABLE
0     WH_ACCT            ACCTNBR    NUMBER              22           0        N
1     WH_ACCT        STMTACCTNBR    NUMBER              22           0        Y
2     WH_ACCT    TAXRPTFORORGNBR    NUMBER              22           0        Y
3     WH_ACCT   TAXRPTFORPERSNBR    NUMBER              22           0        Y
4     WH_ACCT       BRANCHORGNBR    NUMBER              22           0        Y
5     WH_ACCT        NEXTRTXNNBR    NUMBER              22           0        Y
6     WH_ACCT           CURRTERM    NUMBER              22           0        Y
7     WH_ACCT  MAXOVERDRAFTLIMIT    NUMBER              22           3        Y
8     WH_ACCT           NAICSNBR    NUMBER              22           0        Y
9     WH_ACCT        CREDITSCORE    NUMBER              15           0        Y
10    WH_ACCT            ADDRNBR    NUMBER              22           0        Y


,TABLE_NAME,COLUMN_NAME,DATA_TYPE,DATA_PRECISION,DATA_SCALE,NULLABLE
0,WH_ACCT,ACCTNBR,NUMBER,22,0,N
1,WH_ACCT,STMTACCTNBR,NUMBER,22,0,Y
2,WH_ACCT,TAXRPTFORORGNBR,NUMBER,22,0,Y
3,WH_ACCT,TAXRPTFORPERSNBR,NUMBER,22,0,Y
4,WH_ACCT,BRANCHORGNBR,NUMBER,22,0,Y
5,WH_ACCT,NEXTRTXNNBR,NUMBER,22,0,Y
6,WH_ACCT,CURRTERM,NUMBER,22,0,Y
7,WH_ACCT,MAXOVERDRAFTLIMIT,NUMBER,22,3,Y
8,WH_ACCT,NAICSNBR,NUMBER,22,0,Y
9,WH_ACCT,CREDITSCORE,NUMBER,15,0,Y


## 7. EXTREME NUMERIC VALUES
Check for values that might overflow Decimal types

In [21]:
numeric_cols_loans = ['ORIGBAL', 'NOTEBAL', 'BOOKBALANCE', 'AVAILBALAMT', 'COBAL', 'PCTPARTSOLD', 'LCRATE', 'OLDPI', 'PF']

print("WH_LOANS - NUMERIC VALUE RANGES")
print("="*70)

for col in numeric_cols_loans:
    try:
        df = pd.read_sql(f"""
            SELECT 
                '{col}' as column_name,
                MIN({col}) as min_val,
                MAX({col}) as max_val,
                COUNT(*) as non_null_count
            FROM COCCDM.WH_LOANS
            WHERE {col} IS NOT NULL
        """, conn)
        print(f"{col}: min={df['MIN_VAL'].iloc[0]}, max={df['MAX_VAL'].iloc[0]}, count={df['NON_NULL_COUNT'].iloc[0]}")
    except Exception as e:
        print(f"{col}: ERROR - {e}")

WH_LOANS - NUMERIC VALUE RANGES


C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\11944661.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"""


ORIGBAL: min=0, max=65500000, count=114709485
NOTEBAL: ERROR - Execution failed on sql '
            SELECT 
                'NOTEBAL' as column_name,
                MIN(NOTEBAL) as min_val,
                MAX(NOTEBAL) as max_val,
                COUNT(*) as non_null_count
            FROM COCCDM.WH_LOANS
            WHERE NOTEBAL IS NOT NULL
        ': ORA-00904: "NOTEBAL": invalid identifier
Help: https://docs.oracle.com/error-help/db/ora-00904/
BOOKBALANCE: ERROR - Execution failed on sql '
            SELECT 
                'BOOKBALANCE' as column_name,
                MIN(BOOKBALANCE) as min_val,
                MAX(BOOKBALANCE) as max_val,
                COUNT(*) as non_null_count
            FROM COCCDM.WH_LOANS
            WHERE BOOKBALANCE IS NOT NULL
        ': ORA-00904: "BOOKBALANCE": invalid identifier
Help: https://docs.oracle.com/error-help/db/ora-00904/
AVAILBALAMT: min=-1429487, max=30141470, count=6194137
COBAL: ERROR - Execution failed on sql '
            SELECT

## 8. ALL DATE COLUMNS - FULL SCHEMA CHECK

In [22]:
run_query("""
SELECT table_name, column_name, data_type
FROM all_tab_columns
WHERE owner = 'COCCDM'
  AND table_name IN ('WH_LOANS', 'WH_ACCTCOMMON', 'WH_ACCT')
  AND data_type = 'DATE'
ORDER BY table_name, column_id
""", "ALL DATE COLUMNS IN TARGET TABLES")


ALL DATE COLUMNS IN TARGET TABLES
       TABLE_NAME             COLUMN_NAME DATA_TYPE
0         WH_ACCT                 RUNDATE      DATE
1         WH_ACCT         DATELASTCONTACT      DATE
2         WH_ACCT                 DATEMAT      DATE
3         WH_ACCT            CONTRACTDATE      DATE
4         WH_ACCT           DATELASTMAINT      DATE
5         WH_ACCT               SCOREDATE      DATE
6         WH_ACCT                OPENDATE      DATE
7   WH_ACCTCOMMON                 EFFDATE      DATE
8   WH_ACCTCOMMON     CURRACCTSTATEFFDATE      DATE
9   WH_ACCTCOMMON  NOTENEXTRATECHANGEDATE      DATE
10  WH_ACCTCOMMON                 DATEMAT      DATE
11  WH_ACCTCOMMON            CONTRACTDATE      DATE
12  WH_ACCTCOMMON               CLOSEDATE      DATE
13  WH_ACCTCOMMON           DATELASTMAINT      DATE
14       WH_LOANS                 RUNDATE      DATE
15       WH_LOANS             NEXTRATECHG      DATE
16       WH_LOANS          LASTPMTCHGDATE      DATE
17       WH_LOANS          LA

C:\Users\w3221365\AppData\Local\Temp\ipykernel_30772\2064764766.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,TABLE_NAME,COLUMN_NAME,DATA_TYPE
0,WH_ACCT,RUNDATE,DATE
1,WH_ACCT,DATELASTCONTACT,DATE
2,WH_ACCT,DATEMAT,DATE
3,WH_ACCT,CONTRACTDATE,DATE
4,WH_ACCT,DATELASTMAINT,DATE
5,WH_ACCT,SCOREDATE,DATE
6,WH_ACCT,OPENDATE,DATE
7,WH_ACCTCOMMON,EFFDATE,DATE
8,WH_ACCTCOMMON,CURRACCTSTATEFFDATE,DATE
9,WH_ACCTCOMMON,NOTENEXTRATECHANGEDATE,DATE


## 9. SUMMARY

In [23]:
print("\n" + "="*70)
print("DIAGNOSTIC SUMMARY")
print("="*70)
print(f"\nRun completed at: {datetime.now()}")
print("\nNext steps:")
print("1. Review any columns with INVALID rows above")
print("2. Check NUMBER columns with NULL precision (unlimited)")
print("3. Use CASE WHEN in CopyJob query to handle bad dates")
print("\nPush this notebook output back to the repo for analysis.")


DIAGNOSTIC SUMMARY

Run completed at: 2025-12-02 20:19:41.982453

Next steps:
1. Review any columns with INVALID rows above
2. Check NUMBER columns with NULL precision (unlimited)
3. Use CASE WHEN in CopyJob query to handle bad dates

Push this notebook output back to the repo for analysis.


In [ ]:
# Close connection
conn.close()
print("Connection closed.")